# IEMS308 HW2 Association Rules

Author: Taige Hong

In [366]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import csv
from apyori import apriori

stri = pd.read_csv("Dillards POS/strinfo.csv", header = None)

I am using the data from all Dillard's POS data from three locations in Illinois to analyze the association rule. This is based on the assumption that people in the same state share similar purchasing patterns. This process is easy to duplicate with other states as input.

In [76]:
stri[stri.state == "IL"]

,store,name,state,zip,extra
42,603,FAIRVIEW HEIGHTS,IL,62208,1
64,1003,MARION,IL,62959,1
251,4903,MOLINE,IL,61265,1


In [193]:
stri.columns = ['store', 'name', 'state', 'zip', 'extra']

In [195]:
trns = pd.read_csv("Dillards POS/trnsact.csv", header = None)
trns.columns = ['sku', 'store', 'register', 'trannum', 'seq', 'saledate', 'stype', 'quantity', 'retail', 'orgprice', 'amt', 'interid' ,'mic', 'extra']
trns = trns[trns.store.isin([603,1003,4903])][["sku", "store", "register", "trannum", "saledate", "stype", "quantity", "orgprice"]]
trns = trns[trns.stype == "P"]

In [196]:
trns.head(10)

,sku,store,register,trannum,saledate,stype,quantity,orgprice
327,164,603,330,3900,2005-08-13,P,1,20.00
409,164,4903,480,1300,2005-08-05,P,1,20.00
686,268,603,60,1400,2005-02-21,P,1,24.12
694,268,1003,13,94600,2004-11-04,P,1,36.00
1798,450,603,580,1000,2005-08-01,P,1,3.00
1799,450,603,580,1800,2005-08-03,P,1,3.00
1837,450,1003,190,2200,2005-06-16,P,1,6.00
1838,450,1003,660,2800,2005-07-13,P,1,3.99
1839,450,1003,700,400,2005-08-25,P,1,2.10
2337,450,4903,520,4000,2005-07-31,P,1,3.00


I create a new column called entry to identify which SKUs are in the same basket. In fact, this column serves as the ID of the basket.

In [197]:
trns["entry"] = trns["store"].astype(str) + trns["register"].astype(str) + trns["trannum"].astype(str) + trns["saledate"].astype(str)

There are 371,353 distinct baskets.

In [198]:
len(trns["entry"].value_counts())

371353

In this case, I'm setting each individual's minimum support to be 0.01%, which requires a single SKU to appear in more than 37 rows to be considered.

In [205]:
skuoccurence = trns.sku.value_counts()
trns1 = trns[trns["sku"].isin(np.array(skuoccurence[skuoccurence > 37].index))]

In [206]:
trns1.head(10)

,sku,store,register,trannum,saledate,stype,quantity,orgprice,entry
66138,7915,603,80,800,2005-03-11,P,1,19.5,603808002005-03-11
66139,7915,603,80,800,2005-03-13,P,1,19.5,603808002005-03-13
66140,7915,603,80,2600,2005-02-27,P,1,19.5,6038026002005-02-27
66141,7915,603,190,700,2005-07-02,P,1,19.5,6031907002005-07-02
66142,7915,603,240,200,2005-01-22,P,1,19.5,6032402002005-01-22
66143,7915,603,240,500,2005-05-17,P,1,19.5,6032405002005-05-17
66144,7915,603,240,700,2004-11-07,P,1,19.5,6032407002004-11-07
66145,7915,603,240,900,2004-12-04,P,1,19.5,6032409002004-12-04
66147,7915,603,240,1100,2005-07-06,P,1,19.5,60324011002005-07-06
66148,7915,603,240,1100,2005-08-16,P,1,19.5,60324011002005-08-16


62511 baskets remain.

In [392]:
len(trns1["entry"].value_counts())

62511

In [219]:
entrylist = trns1.entry.unique()

In [230]:
records = []
for i in range(len(entrylist)):
    trns2 = trns1[trns1["entry"] == entrylist[i]]
    records.append(list(trns2.sku.unique()))

I'm using the apriori function from apyori package to calculate the support, confidence and lift. The parameters are set by trials and errors method.

In [350]:
ar = apriori(records, min_support = 0.0002, min_confidence = 0.02, min_lift = 2, min_length = 2)

In [351]:
result = list(ar)

243 rules are generated. However, our objective is to find 100 SKUs to move, so we need to eliminate the duplicate candidates and find the best 100.

In [394]:
len(result)

243

In [354]:
top100 = output.sort_values("lift", ascending = False).drop_duplicates(subset = "analysis").head(100)

In [364]:
top100['result'] = top100['analysis'].astype(str) + "=>" + top100['association'].astype(str)

The result is stored in the 'result.csv' file in the same folder.

In [389]:
with open('result.csv', 'w'):
    writer = csv.writer(open('result.csv', 'w'))
    writer.writerow(tuple(top100['result']))

The support here is relatively low because ~70% of the baskets contain only one item. This fact is inflating the lift and deflating the support.

In [400]:
1 - sum(len(records[i]) > 1 for i in range(len(records)))/len(records)

0.6956695621570603

In [427]:
sku = pd.read_csv("Dillards POS/skuinfo.csv", header = None, usecols = [0,1,9], error_bad_lines = False)
dept = pd.read_csv("Dillards POS/deptinfo.csv", header = None)

Result mapped to their brand names:

In [430]:
sku[sku[0].isin(list(top100.analysis))][9].value_counts()

NOBLE CH     15
SUMMER S     12
NOBLE EX     12
GREAT AM     10
MILCO IN      7
CLINIQUE      6
CABERNET      5
CROSCILL      5
WESTPOIN      4
HUE/KAYS      4
HUE           4
NUTMEG M      3
NOBILITY      2
CAPELLI       2
LANCOME       2
ROYCE HO      2
CHRISTY       2
MAIN ING      1
SATURDAY      1
SAN FRAN      1
Name: 9, dtype: int64

In [431]:
sku[sku[0].isin(list(top100.association))][9].value_counts()

NOBLE EX     12
NOBLE CH     11
SUMMER S     10
GREAT AM      8
MILCO IN      7
CLINIQUE      6
CABERNET      5
CROSCILL      4
HUE           4
HUE/KAYS      3
NUTMEG M      2
WESTPOIN      2
CAPELLI       2
LANCOME       2
ROYCE HO      2
NOBILITY      1
MAIN ING      1
SATURDAY      1
CHRISTY       1
SAN FRAN      1
Name: 9, dtype: int64